In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
def getSoup(url):
    r = requests.get(url)
    html_doc = r.content
    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup

In [3]:
url_base="https://consultation.ngi.eu"
channels_url = '/channels'

In [4]:
url = url_base + channels_url
soup = getSoup(url)

In [5]:
pages = soup.find_all("li",class_="pager-next last")[0].find_all('a')[0].attrs['href'].split("?page=")[1]

In [6]:
channels_data = []
comments_data = []
posts_data = []
for page in range(int(pages)+1):
    url = url_base + channels_url + "?page=" + str(page)
    channels = soup.find_all("a", class_="btn btn-primary")
    for channel in channels:
        if channel.get_text() == 'Read more ':
            channel_url = channel.attrs['href']
            channel_url = url_base + channel_url  
            soup = getSoup(channel_url)
            channel_pages = 0
            title_channel = soup.find_all("h1",id="page-title")[0].get_text().replace("\t","").replace("\n","").strip()
            if len(soup.find_all("li",class_="pager-last last")) > 0:
                channel_pages = int(soup.find_all("li",class_="pager-last last")[0].find_all('a')[0].attrs['href'].split("?page=")[1])
            channel_pages += 1
            channel_data = {}
            channel_data['title'] = title_channel
            channel_data['url'] = channel_url
            channels_data.append(channel_data)
            for cp in range(channel_pages):
                url_channel = channel_url + "?page=" + str(cp)
                soup = getSoup(url_channel)
                channel_posts = soup.find_all("", class_="blog-content-tile")
                for idx_post in range(len(channel_posts)):
                    post_data = {}
                    url_post = url_base + channel_posts[idx_post].find_all('a')[0].attrs['href']
                    soup = getSoup(url_post)
                    title = soup.find_all("h1",id="page-title")[0].get_text().replace("\n","").replace("\t","").strip()
                    username = soup.find_all("article")[0].find_all("div",class_="username")[0].get_text().replace("\n","").replace("\t","").strip()
                    url_username = soup.find_all("article")[0].find_all("div",class_="username")[0].find_all("a")[0].attrs['href']
                    text = soup.find_all("article")[0].find_all("div",class_="field-item even")[0].text
                    month = soup.find_all("article")[0].find_all("span",class_="month")[0].get_text()
                    day = soup.find_all("article")[0].find_all("span",class_="day")[0].get_text()
                    year = soup.find_all("article")[0].find_all("span",class_="year")[0].get_text()
                    post_date = day + " " + month + " " + year
                    comments = soup.find_all("section",id="comments")
                    post_data["title"] = title
                    post_data['username'] = username
                    post_data['url_username'] = url_base + url_username
                    post_data['url'] = url_post
                    post_data['text'] = text
                    post_data['date'] = post_date
                    post_data['channel'] = title_channel
                    posts_data.append(post_data)
                    if len(comments) >= 1:
                        for idx_comment in range(len(comments)+1):
                            idx_comment = idx_comment -1
                            comment_data = {}
                            user_comment_avatar = url_base +comments[idx_comment].find_all("div",class_="user-picture")[0].find_all("a")[0].find_all("img")[0].attrs["src"]
                            user_comment_name = comments[idx_comment].find_all("div",class_="username")[0].get_text().replace("\n","").replace("\t","").strip()
                            user_comment_profile = url_base + comments[idx_comment].find_all("div",class_="user-picture")[0].find_all("a")[0].attrs["href"]
                            user_comment_text = comments[idx_comment].find_all("div",class_="comment-body")[0].text
                            user_comment_date = comments[idx_comment].find_all("div",class_="date")[0].get_text().replace("\n","").replace("\t","").replace("answered ","").strip()
                            comment_data['channel'] = title_channel
                            comment_data['post'] = title
                            comment_data['url_post'] = url_post
                            comment_data['date'] = user_comment_date
                            comment_data['username'] = user_comment_name
                            comment_data['username_profile'] =  user_comment_profile
                            comment_data['text'] = user_comment_text
                            comment_data['username_avatar'] = user_comment_avatar
                            comments_data.append(comment_data)
            

In [7]:
df_posts = pd.DataFrame(posts_data)

In [8]:
df_comments = pd.DataFrame(comments_data)

In [9]:
df_channels = pd.DataFrame(channels_data)

In [10]:
df_posts.to_json("posts_data.json")

In [11]:
df_posts.to_csv("posts_data.csv",index=False)

In [12]:
df_comments.to_json("comments_data.json")

In [13]:
df_comments.to_csv("comments_data.csv",index=False)

In [14]:
df_channels.to_json("channels_data.json")

In [15]:
df_channels.to_csv("channels_data.csv")